In [1]:
import pandas as pd

In [50]:
cashier = pd.read_excel("data\Продажи по кассам.xlsx")
reference = pd.read_excel("data\Справочник касс.xlsx")
cashier.head()

,cash_id,cash_name,sale_sum
0,5,ККМ №6 (Кофемат),199.98
1,6,КСО2,209807.91
2,7,КСО1,112335.55
3,8,КСО2,71465.34
4,9,КСО3,29865.31


In [51]:
reference

,cash_id,cash_name,store_id,store_name
0,1,ККМ №23 Vmcash,183,Командор №72
1,2,ККМ №2,450,Командор №338
2,3,ККМ №51 Vmcash,124,Командор №51
3,4,ККМ №3,393,Командор №297
4,5,ККМ №6 (Кофемат),199,Командор №116
...,...,...,...,...
3583,3584,ККМ №11 (Терминал),160,Командор №81
3584,3585,КСО №2,608,Командор №484
3585,3586,ККМ №1,608,Командор №484
3586,3587,КСО №3,608,Командор №484


In [52]:
reference.head()

,cash_id,cash_name,store_id,store_name
0,1,ККМ №23 Vmcash,183,Командор №72
1,2,ККМ №2,450,Командор №338
2,3,ККМ №51 Vmcash,124,Командор №51
3,4,ККМ №3,393,Командор №297
4,5,ККМ №6 (Кофемат),199,Командор №116


In [53]:
cashier.duplicated(subset=["cash_id"]).any()

np.False_

In [54]:
reference.duplicated(subset=["cash_id"]).any()

np.False_

Дубликатов нет. Колонку cash_id сделаем индексом

In [55]:
cashier.set_index("cash_id", inplace=True)
reference.set_index("cash_id", inplace=True)


In [56]:
joined = cashier.join(reference, on="cash_id", lsuffix='_cash', rsuffix='_ref')

In [57]:
joined

,cash_name_cash,sale_sum,cash_name_ref,store_id,store_name
cash_id,,,,,
5,ККМ №6 (Кофемат),199.98,ККМ №6 (Кофемат),199,Командор №116
6,КСО2,209807.91,КСО2,260,Командор №172
7,КСО1,112335.55,КСО1,7,Командор №4
8,КСО2,71465.34,КСО2,7,Командор №4
9,КСО3,29865.31,КСО3,7,Командор №4
...,...,...,...,...,...
2652,КСО №2,63565.38,КСО №2,75,Командор №27
2653,КСО3,50076.60,КСО3,181,Командор №102
2654,КСО2,34063.47,КСО2,181,Командор №102


### Агрегация

In [93]:
aggregated = joined.groupby(["store_id"]).agg({
    "sale_sum": "sum",
}).round(2)
aggregated.head()


,sale_sum
store_id,
3,592471.59
5,1129652.40
7,989422.15
16,770074.33
22,6842874.38


In [94]:
aggregated.iloc[9]

sale_sum    885658.82
Name: 48, dtype: float64

In [85]:
store = pd.read_excel("data\Продажи по магазинам.xlsx", index_col="store_id")
store.head()

,store_name,sale_sum
store_id,,
3,Командор №2,592471.59
5,Командор №3,1129652.40
7,Командор №4,989422.15
16,Командор №6,770074.33
22,Командор №7,6842874.38


In [78]:
len(store) == len(aggregated)

True

In [101]:
diff_mask = aggregated["sale_sum"] != store["sale_sum"]
mismatched = aggregated[diff_mask].copy()
mismatched["store_sale_sum"] = store.loc[diff_mask, "sale_sum"]
mismatched[["sale_sum", "store_sale_sum"]]

,sale_sum,store_sale_sum
store_id,,
90,984625.66,984825.66
144,1083058.49,1083090.49


In [102]:

common_index = aggregated.index.intersection(store.index)